In [1]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
import time

C:\Users\srila\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
movies='movies.csv'
ratings='ratings.csv'

df_movies=pd.read_csv(movies, usecols=['movieId','title'], dtype={'movieId':'int32','title':'str'})
df_ratings=pd.read_csv(ratings, usecols=['userId', 'movieId', 'rating'], dtype={'userId':'int32','movieId':'int32','rating':'float32'})

In [3]:
#Sparse Matrix

movies_users=df_ratings.pivot(index='movieId', columns='userId', values='rating').fillna(0)
mat_movies_users=csr_matrix(movies_users.values)

In [4]:
model_knn=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=20)

In [5]:
model_knn.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [6]:
# Recommender (movie_name) => List of Movies recommended
def recommender(movie_name, data, model, n_recommendations):
    print('Searching Movie Recommendations....')
    time.sleep(2)
    model.fit(data)
    idx=process.extractOne(movie_name, df_movies['title'])[2]
    
    distances, indices = model.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    raw_recommends = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1] 
    moviesList = [x for x in df_movies['title']]
    
    recommendations = []
    
    for i, (idx, dist) in enumerate(raw_recommends):
        recommendations.append(moviesList[idx])
        
    recommendations.reverse()
    
    print('\n People who like {} may also like the following:\n'.format(movie_name))
    for i, x in enumerate(recommendations):
        print(i+1, ': ',x)


In [10]:
#driver function
movieName = input('Enter a movie name: ')
recommender(movieName, mat_movies_users, model_knn, 20)

Enter a movie name: Aladdin
Searching Movie Recommendations....

 People who like Aladdin may also like the following:

1 :  Beauty and the Beast (1991)
2 :  Lion King, The (1994)
3 :  Jurassic Park (1993)
4 :  True Lies (1994)
5 :  Batman (1989)
6 :  Ace Ventura: Pet Detective (1994)
7 :  Mrs. Doubtfire (1993)
8 :  Die Hard: With a Vengeance (1995)
9 :  Batman Forever (1995)
10 :  Apollo 13 (1995)
11 :  Mask, The (1994)
12 :  Speed (1994)
13 :  Forrest Gump (1994)
14 :  Fugitive, The (1993)
15 :  Dumb & Dumber (Dumb and Dumber) (1994)
16 :  While You Were Sleeping (1995)
17 :  Toy Story (1995)
18 :  GoldenEye (1995)
19 :  Babe (1995)
20 :  Pretty Woman (1990)
